# Benchmark Muscato Alignment Software

Here are the various data and scripts needed to reproduce my benchmarking of the Muscato software. 
Bowtie2 was used as a long standing quality general RNA-seq aligner that has been used for metatranscriptomics.

### Data and Software

Data: NCBI Bacterial 16S rRNA genes were downloaded on 4/27/18 from: ftp://ftp.ncbi.nlm.nih.gov/refseq/TargetedLoci/Bacteria/<br>
<br>Software:<br>
Bowtie version: bowtie2/2.3.3<br>
Go version: go/1.10.1<br>
<br>
EmEditor was used to:<br>
- retrieve taxon IDs from the .gbff file
- flatten the .fna file into a tab delimited .txt file
- in conjunctoin with excel "vlookup" modify the gene names with the taxon id
- remove sequences  < 500nt
- deduplicate the sequences<br>
- remove any sequences with ambiguous bases using standard regex.

### Make Reads

#### NCBI16SRRNA.txt

#### Make_16S_Reads.pl

This script creates either 20K reads of lengths 50, 75, 100, and 250 with 0, 1, 2, or 3 mismatches between the read and source gene.

In [ ]:
#MAKE 20K reads
@LENS = (50,75,100,250);
@MIS = (0,1,2,3);

foreach my $rlen (@LENS){
	foreach my $mism (@MIS){

		$outfq = $rlen."_".$mism."_reads.fastq";
		open(OUTFQ, ">", $outfq)||die;
		$outfa = $rlen."_".$mism."_reads.fasta";
		open(OUTFA, ">", $outfa)||die;

		$rc = 0;
		my %READS;
		while($rc < 20000){
			
			$ingen = 'NCBI16SRRNA.txt';
			open(INGEN, $ingen)||die;
			while(<INGEN>){
				
				$_=~s/[\r\n]+//;
				if($_ !~ /\w/){next;}
				$_=uc($_);
				(my $gname, my $gene) = split('\t',$_);
				if($gene =~ /[^ATGC]/ || length($gene)<500 || length($gene)>2500){next;}
				
				#make read
				$glen = length($gene);
				for $x (0..$glen-1){
						$start = int(rand(length($gene-$rlen+1)));
						$read = substr($gene, $start, $rlen);
						if($mism > 0){
							$done = '';
							for my $i (1..$mism){
								$mpos = int(rand($rlen));
								while($done =~ /\b$mpos\b/){$mpos = int(rand($rlen));}
								$done.="|".$mpos;
								$bb = substr($read, $mpos, 1);
								if($bb eq "A"){$bb = "G";}
								elsif($bb eq "T"){$bb = "C";}
								elsif($bb eq "G"){$bb = "T";}
								else{$bb = "A";}
								substr($read, $mpos, 1, $bb);
							}
						}
						$SCORE = ("I" x $rlen);
						$out = $rc."_".$gname;
						if(!exists($READS{$read})){$READS{$read}=1; last;}
				}
				
				print OUTFA ">".$out."\n$read\n";
				print OUTFQ "@".$out."\n$read\n+\n$SCORE\n";
				
				$rc++;
				if($rc%1000==0){ print "on $rc mism $mism len $rlen out $out read $read\n";} 
				$on++; 				
				if($rc == 20000){last;}

			}
		}
	}
}

#### Make_16S_Big_Reads.pl

This script creates \$count specified unique reads between the lengths of \$min to \$max with 0, 1, 2, or 3 mismatches.

In [ ]:
$min = 75;
$max = 100;
$count = 1000000;

$ingen = 'NCBI16SRRNA.txt';  #gene database tab delim name\tseq

$outfq = '16S_'.$count.'_reads.fastq';
open(OUTFQ, ">", $outfq)||die;
$outfa = '16S_'.$count.'_reads.fasta';
open(OUTFA, ">", $outfa)||die;

$rc = 0;
@MIS = (0,1,2,3);
$nm = @MIS;
my %READS; 
while($rc < $count){
	open(INGEN, $ingen)||die;
	while(<INGEN>){
		$_=~s/[\r\n]+//;
		if($_ !~ /\w/){next;}
		$_=uc($_);
		(my $gname, my $gene) = split('\t',$_);
		if($gene =~ /[^ATGC]/ || length($gene)<500 || length($gene)>2500){next;}
		
		#make read
		$glen = length($gene);
		for $x (0..$glen-1){
				$mism = $MIS[int(rand($nm))];
				$rlen = int(rand($max-$min+1))+$min;
				$start = int(rand(length($gene-$rlen)));
				$read = substr($gene, $start, $rlen);
				if($mism > 0){
					$done = '';
					for my $i (1..$mism){
						$mpos = int(rand($rlen));
						while($done =~ /\b$mpos\b/){$mpos = int(rand($rlen));}
						$done.="|".$mpos;
						$bb = substr($read, $mpos, 1);
						if($bb eq "A"){$bb = "G";}
						elsif($bb eq "T"){$bb = "C";}
						elsif($bb eq "G"){$bb = "T";}
						else{$bb = "A";}
						substr($read, $mpos, 1, $bb);
					}
				}
				$SCORE = ("I" x $rlen);
				$out = $rc."_".$gname;
				if(!exists($READS{$read})){$READS{$read}=1; last;}
		}

		print OUTFA ">".$out."\n$read\n";
		print OUTFQ "@".$out."\n$read\n+\n$SCORE\n";
		
		$rc++;
		if($rc%10000==0){ print "on $rc mism $mism len $rlen out $out read $read\n";} 
		$on++; 				
		if($rc == $count){last;}
	}
}


#### 16S_1000000_reads.fastq

### Run Bowtie

#### BOWTIE_COUNTS.pl

In [ ]:
use warnings;
$input   = './16S_SAMPLE_MAXI.bowtie';
$inlin   = './NCBI16S_LINEAGES.txt';
$outstat = $input;
$outstat =~ s/\.bowtie$/.stats/;
		
$mmtol = 1;
$cut = 0.06; #94% id
			
#INPUT LINEAGES
open(INLIN, $inlin)||die;
while(<INLIN>){
	$_ =~ s/[\n\r]+//;
	if($_ !~ /\w/){next;}
	$_ = uc($_);
	@stuff = split("\t", $_);
	$TID = shift(@stuff);
	$lineage = join(";", @stuff); 
	$PHY{$TID}=$lineage;
}


#PROCESS READS
$on=0;
$time = localtime;
print "Inputting reads time $time\n";
open(INMATCH, $input)||die;
while(<INMATCH>){
			if($_ !~ /\w/){next;}
			$_ =~ s/[\n\r\>]+//g;
			@stuff = split("\t", $_);
			if($stuff[1] !~ /\b(0|256)\b/){next;}
			
			#do initial screening of mismatches to reduce total matches
			$_ =~ /NM:i:(\d)+/;
			$MISM = $1; 
			$TOT{$stuff[0]}++;
			if($MISM > length($stuff[9])*$cut){next;}
			if(!exists($BEST{$stuff[0]})){$BEST{$stuff[0]}=$MISM;}
			if($MISM>$BEST{$stuff[0]}+$mmtol){next;}
			if($MISM<$BEST{$stuff[0]}){$BEST{$stuff[0]}=$MISM;}

			$MATCHES{$stuff[0]}{$stuff[2]}=$MISM;
			if($on%100000==0){
				$time = localtime;
				print "on $on mism $MISM best $BEST{$stuff[0]} read $stuff[0] time $time\n";}
			$on++;
}


#PROCESS READS FOR BEST MATCHES
$time = localtime;
print "Cleaning reads time $time\n";
$on=0;
foreach my $read (keys %MATCHES){
		$best = $BEST{$read};
		$KC = 0;
		foreach my $gene (keys %{$MATCHES{$read}}){
			if($MATCHES{$read}{$gene} > $best+$mmtol){ delete($MATCHES{$read}{$gene}); }
			else{$KC++;}
		}
		$BEST{$read}=$KC;
		
		if($on%100000==0){print "on $on best mism $MISM tc $TOT{$read} kc $KC \n";}
		$on++;
}


#GET LCA and STATS
$time = localtime;
print "Processing read LCA and stats $outstat time $time\n";
open(OUTSTAT, ">", $outstat)||die;
print OUTSTAT "Read Name\tMatches contained gene ID\tLCA matches Read Lineage\tKept Gene Matches\tTotal Gene Matches\tRead Lineage\tLCA\tHas Kingtom\tHas Phylum\tHas Class\tHas Order\tHas Family\tHas Genus\tHas Species\n";
foreach my $read (keys %MATCHES){
 
	$read =~ /\|(\d+)\|/;
	$RTID = $1;
	$RLIN = $PHY{$RTID};
	$read =~ /^[\d\_]+([^\|]+)\|/;
	$RGID = $1;

	#get good hits
	my @PHYL;
	my @MNAMES;
	foreach my $gene (keys %{$MATCHES{$read}}){
		$gene =~/\|(\d+)\|/; 
		$GTID = $1;
		$GLIN = $PHY{$GTID};
		$gene =~ /^([^\|]+)\|/;
		$GGID = $1;
		push(@PHYL, $GLIN);
		push(@MNAMES, $GGID);
	}

	#get common ancestor
	my @TAXON;
	my %seen3;
	@PHYL = grep { !$seen3{$_}++ } @PHYL;
	$len1 = @PHYL; 
	if($len1 == 1){$LCA = $PHYL[0];	@TAXON = split(";", $LCA);}
	elsif($len1 >1){
		$first = $PHYL[0];
		@levels = split(";", $first);
		for my $p (0..$#levels){
			@matched = grep(/(^\Q$levels[$p]\E\;|\;\Q$levels[$p]\E\;|\;\Q$levels[$p]\E$)/i, @PHYL);  
			$len2 = @matched; 
			if($len2 == $len1){push(@TAXON, $levels[$p]);}
			else{last;}
		}
		$len3 = @TAXON; 
		if($len3 > 1){$LCA = join(";", @TAXON);}
		elsif($len3==1){$LCA = $TAXON[0];}
		else{$LCA = "NCA"; @TAXON = split(";", $LCA);}}
	else{$LCA = "NCA"; @TAXON = split(";", $LCA);}


	#check accuracy
	$hasGID = 0;
	if( grep{$RGID eq $_} @MNAMES){$hasGID = 1;}
	$hasLIN = 0;
	if( $RLIN =~ /$LCA/i ){ $hasLIN =1;}
	
	my @GLCA;
	foreach(@TAXON){
		if($RLIN =~ /$_/){push(@GLCA,1);}
		else{push(@GLCA,-1);}
	}
	$arlen = @GLCA;
	while($arlen < 7){ push(@GLCA,0); $arlen = @GLCA;}
	$wasgoodlin = join("\t", @GLCA);
	print OUTSTAT "$read\t$hasGID\t$hasLIN\t$BEST{$read}\t$TOT{$read}\t$RLIN\t$LCA\t$wasgoodlin\n";
}

#### bash BOWTIE_TESTS.sh >bowtie.log 2>&1

Runs the bowtie tests on the 4 lengths and 4 mismatches and 5 maxmatches. Output is stored in bowtie.log. Times are stored in bowtie_times.txt

In [ ]:
#bash sccript
#do very-fast tests
while read p; 
	do echo $p; 
	while read q; 
		do echo -ne "$p start $q $(date)" >> bowtie_times.txt;
		cp BOWTIE.sh ${q}.sh; 
		sed -i "s|MAXI|$p|g" ${q}.sh;
		sed -i "s|SAMPLE|$q|g" ${q}.sh;
		bash ${q}.sh;
		echo -ne " finalign $(date)" >> bowtie_times.txt;
		cp BOWTIE_COUNTS.pl ${q}.pl;
		sed -i "s|MAXI|$p|g" ${q}.pl; 
		sed -i "s|SAMPLE|$q|g" ${q}.pl; 
		perl ${q}.pl;
 		echo " done $(date)" >> bowtie_times.txt;
	done <samplist.txt;  
done <multimap.txt

sed -i 's/fast/sensitive/' BOWTIE.sh
sed -i 's/SAMPLE_MAXI/SAMPLEvs_MAXI/' BOWTIE.sh
sed -i 's/SAMPLE_MAXI/SAMPLEvs_MAXI/' BOWTIE_COUNTS.pl

#do very-sensitive tests
while read p; 
	do echo $p; 
	while read q; 
		do echo -ne "$p start $q $(date)" >> bowtie_times.txt;
		cp BOWTIE.sh ${q}.sh; 
		sed -i "s|MAXI|$p|g" ${q}.sh;
		sed -i "s|SAMPLE|$q|g" ${q}.sh;
		bash ${q}.sh;
		echo -ne " finalign $(date)" >> bowtie_times.txt;
		cp BOWTIE_COUNTS.pl ${q}.pl;
		sed -i "s|MAXI|$p|g" ${q}.pl; 
		sed -i "s|SAMPLE|$q|g" ${q}.pl; 
		perl ${q}.pl;
 		echo " done $(date)" >> bowtie_times.txt;
	done <samplist.txt;  
done <multimap.txt


#### BOWTIE.sh

bowtie2 --very-fast --norc -k MAXI -t -p 8 --no-hd --no-sq -x NCBI16SRRNA -f 16S_SAMPLE_reads.fasta -S 16S_SAMPLE_MAXI.bowtie

### Run Muscato

Create the full MUSCATO.sh with unix or excel. qsub MUSCATO.pbs. It creates the .stats file for each read type as it runs.

#### MUSCATO.sh

In [ ]:
#repeat for 0, 1, 2, and 3 mismatches x lengths 50, 75, 100 and 250 x 1, 10, 100, 1000, 10000 multimapping 
#50 nt windows:0,5,15,25,34
#75 nt windows:0,15,30,45,59 
#100 nt windows:0,20,40,60,80
#250 nt windows:0,50,100,150,200
#sed the LENGTH, MISMATCHES, and MULTIMAP in unix loop or use excel

echo -ne "start 16S_LENGTH_MISMATCHES_MULTIMAP_matches.txt $(date)" >> musctime.txt
muscato --ReadFileName=16S_LENGTH_MISMATCHES_reads.fastq --ResultsFileName=16S_LENGTH_MISMATCHES_MULTIMAP_matches.txt --GeneFileName=NCBI16SRRNA.txt.sz  --GeneIdFileName=NCBI16SRRNA_ids.txt.sz --Windows=0,20,40,60,80 --WindowWidth=15 --BloomSize=400000000  --NumHash=8  --PMatch=0.94  --MinDinuc=5  --MinReadLength=50 --MaxMatches=MULTIMAP --MaxConfirmProcs=5  --MaxReadLength=250  --MatchMode=best  --MMTol=1 --TempDir=tmp/16S_LENGTH_MISMATCHES_MULTIMAP --SortTemp=tmp/16S_LENGTH_MISMATCHES_MULTIMAP/srt --SortPar=8  --SortMem=50%
echo -ne " finalin $(date)" >> musctime.txt
cp MUSCATO_COUNTS.pl 16S_LENGTH_MISMATCHES_MULTIMAP.pl; sed -i 's/SAMPLE/LENGTH_MISMATCHES_MULTIMAP/g' 16S_LENGTH_MISMATCHES_MULTIMAP.pl; 
perl 16S_LENGTH_MISMATCHES_MULTIMAP.pl
echo " end $(date)" >> musctime.txt

#### MUSCATO.pbs

In [ ]:
#PBS -l nodes=1:ppn=8
#PBS -l pmem=4GB
#PBS -l walltime=24:00:00
#PBS -l qos=flux
#PBS -A andjoh_fluxod
#PBS -q fluxod
#PBS -N muscato_1M1M
#PBS -j oe
#PBS -M tealfurn@umich.edu
#PBS -m abe
#PBS -V

if [ -n "$PBS_O_WORKDIR" ]; then cd $PBS_O_WORKDIR; fi

export LC_ALL=C
export GOPATH=${HOME}/go
export GOBIN=${GOPATH}/bin
export PATH=${GOBIN}:${PATH}
export GOGC=20

bash MUSCATO.sh


#### MUSCATO_COUNTS.pl

In [ ]:
use warnings;
$input   = '16S_SAMPLE_matches_readstats.txt';
$inlin   = 'NCBI16S_LINEAGES.txt';
$outstat = $input;
$outstat =~ s/matches_readstats.txt$/.stats/;
		
#INPUT LINEAGES
open(INLIN, $inlin)||die;
while(<INLIN>){
	$_ =~ s/[\n\r]+//;
	if($_ !~ /\w/){next;}
	$_ = uc($_);
	@stuff = split("\t", $_);
	$TID = shift(@stuff);
	$lineage = join(";", @stuff); 
	$PHY{$TID}=$lineage;
}


#PROCESS READS
$on=0;
$time = localtime;
print "Getting readstats time $time on $outstat\n";
open(OUTSTAT, ">", $outstat)||die;
open(INMATCH, $input)||die;
while(<INMATCH>){	
	if($_ !~ /\w/){next;}
	$_ =~ s/[\n\r\>]+//g;
	(my $read, my $genelist) = split("\t", $_);

	#@reads = split(";", $readlist);
	@genes = split(";", $genelist);

	#foreach my $read (@reads){
		$read =~ /\|(\d+)\|/;
		$RTID = $1;
		$RLIN = $PHY{$RTID};
		$read =~ /^[\@\d\_]+([^\|]+)\|/;
		$RGID = $1;

		my @PHYL;
		my @MNAMES;
		$TGC = @genes;
		foreach my $gene (@genes){
			$gene =~/\|(\d+)\|/; 
			$GTID = $1;
			$GLIN = $PHY{$GTID}; 	
			$gene =~ /^([^\|]+)\|/;
			$GGID = $1;
			push(@PHYL, $GLIN);
			push(@MNAMES, $GGID);
		}


	my @TAXON;
	my %seen3;
	@PHYL = grep { !$seen3{$_}++ } @PHYL;
	$len1 = @PHYL;
	if($len1 == 1){$LCA = $PHYL[0];	@TAXON = split(";", $LCA);}
	elsif($len1 >1){
		$first = $PHYL[0];
		@levels = split(";", $first);
		for my $p (0..$#levels){
			@matched = grep(/(^\Q$levels[$p]\E\;|\;\Q$levels[$p]\E\;|\;\Q$levels[$p]\E$)/i, @PHYL);  
			$len2 = @matched; 
			if($len2 == $len1){push(@TAXON, $levels[$p]);}
			else{last;}
		}
		$len3 = @TAXON; 
		if($len3 > 1){$LCA = join(";", @TAXON);}
		elsif($len3==1){$LCA = $TAXON[0];}
		else{$LCA = "NCA"; @TAXON = split(";", $LCA);}}
	else{$LCA = "NCA"; @TAXON = split(";", $LCA);}

		#check accuracy
		$hasGID = 0;
		if( grep{$RGID eq $_} @MNAMES){$hasGID = 1;}

		$hasLIN = 0;
		if( $RLIN =~ /$LCA/i ){ $hasLIN =1;}
		
		my @GLCA;
		@RLEVS = split(";",$RLIN);
		for my $q (0..6){
			if(!exists($TAXON[$q])){push(@GLCA,0);}
			elsif($RLEVS[$q] eq $TAXON[$q]){push(@GLCA,1);}
			else{push(@GLCA,-1);}
		}

		$wasgoodlin = join("\t", @GLCA);
		print OUTSTAT "$read\t$hasGID\t$hasLIN\t$TGC\t$TGC\t$RLIN\t$LCA\t$wasgoodlin\n";
	#}
}



### Collate Stats

After Bowtie and Muscato are all aligned and have their stats files. Put all the .stats files into the same folder and compile the data using the outstats.pl script. Open outstats.txt in excel or R and graph as you like.

#### outstats.pl

In [ ]:
#use warnings;
#process .o flux output files
$dir = './';
$output = './outstats.txt';

opendir(DIR, $dir) || die "Can't opendir $dir: $!";
foreach my $file (readdir DIR) { 
    	$file = $dir."\\".$file; 
    	if(-f $file && $file =~ /\.stats$/i){
	    	print "doing $file\n";
			push(@files, $file);
			$contGID = 0;
			$LCAmatch = 0;
			$TOTreads =0;
			$MINmism = 0;
			$TOTmatch = 0;
			$GOODmatch = 0;
			$goodKING = 0;
			$noKING = 0;
			$badKING = 0;
			$goodPHYL = 0;
			$noPHYL = 0;
			$badPHYL = 0;
			$goodCLAS = 0;
			$noCLAS = 0;
			$badCLAS = 0;
			$goodORDR = 0;
			$noORDR = 0;
			$badORDR = 0;
			$goodFAMY = 0;
			$noFAMY = 0;
			$badFAMY = 0;
			$goodGENU = 0;
			$noGENU = 0;
			$badGENU = 0;
			$goodSPEC = 0;
			$noSPEC = 0;
			$badSPEC = 0;
			

			open(INPUT, $file)||die "unable to open $file: $!\n";
			while(<INPUT>){

				#for bowtie output
				if($_ !~ /\w/){next;}
				if($_ =~ /^Read/i){next;}
				$_ =~ s/[\n\r]+//;
				@stuff = split("\t", $_);
				
				#get mapping accuracy stats
				$TOTreads++;
				$contGID+=$stuff[1];
				$LCAmatch+=$stuff[2];
				$keptmatch+=$stuff[3];
				$totmatch+=$stuff[4];

				#GET LCA ACCURACY STATS
				if($stuff[7]==1){$goodKING++;}
				if($stuff[7]==0){$noKING++;}
				if($stuff[7]==-1){$badKING++;}
				if($stuff[8]==1){$goodPHYL++;}
				if($stuff[8]==0){$noPHYL++;}
				if($stuff[8]==-1){$badPHYL++;}
				if($stuff[9]==1){$goodCLAS++;}
				if($stuff[9]==0){$noCLAS++;}
				if($stuff[9]==-1){$badCLAS++;}
				if($stuff[10]==1){$goodORDR++;}
				if($stuff[10]==0){$noORDR++;}
				if($stuff[10]==-1){$badORDR++;}
				if($stuff[11]==1){$goodFAMY++;}
				if($stuff[11]==0){$noFAMY++;}
				if($stuff[11]==-1){$badFAMY++;}
				if($stuff[12]==1){$goodGENU++;}
				if($stuff[12]==0){$noGENU++;}
				if($stuff[12]==-1){$badGENU++;}
				if($stuff[13]==1){$goodSPEC++;}
				if($stuff[13]==0){$noSPEC++;}
				if($stuff[13]==-1){$badSPEC++;}
			}
			
		$pctctngid = $contGID/$TOTreads*100;
		$pctlcamat = $LCAmatch/$TOTreads*100;
		$avgkept = $keptmatch/$TOTreads;
		$avgmap = $totmatch/$TOTreads;
		
		$combo = "$file\t$TOTreads\t$pctctngid\t$pctlcamat\t$avgmap\t$avgkept\t$badKING\t$badPHYL\t$badCLAS\t$badORDR\t$badFAMY\t$badGENU\t$badSPEC\t$goodKING\t$goodPHYL\t$goodCLAS\t$goodORDR\t$goodFAMY\t$goodGENU\t$goodSPEC\t$noKING\t$noPHYL\t$noCLAS\t$noORDR\t$noFAMY\t$noGENU\t$noSPEC";
		print "$combo\n";
		push(@RESULTS, $combo);

	}
}

open(OUTPUT, ">>", $output)||die;
foreach(@RESULTS){
	print OUTPUT "$_\n";
}